In [ ]:
__nbid__ = '0078'
__author__ = 'Brian Merino <brian.merino@noirlab.edu>'
__version__ = '20260121' # yyyymmdd; version datestamp of this notebook
__datasets__ = ['smash_dr2', 'des_dr2', 'decaps_dr2']
__keywords__ = ['AladinLite', 'ipyaladin']

# Generating MOCs with Aladin Lite v3

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [Start Aladin viewer](#Aladin)
* [Accessing the data](#Data)
* [Querying the Data Lab database](#Query)
* [Prepare MOCs](#prepare)
* [Plot the MOCs](#Plot)
* [Add MOCs to Aladin](#AladinMOCs)
* [Add individual sources to Aladin](#source)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to query the Data Lab via the query client service and use the data to create Multi-Order Coverage maps (MOCs) that will then be displayed with the Aladin Lite viewer.


<a class="anchor" id="summary"></a>
# Summary
<a href="https://aladin.cds.unistra.fr/AladinLite/doc/">AladinLite</a> is an interactive sky atlas that runs in your browser. Aladin allows users to explore the sky and display images. Additionally, it enables the overlay of catalog objects, facilitating object identification. <a href="https://github.com/cds-astro/ipyaladin">ipyaladin</a> was created to allow Jupyter Notebooks to utilize Aladin Lite's functionality. This notebook will demonstrate how to use ipyaladin to display images and overlay survey catalog data, which would help identify overlapping datasets. 

Visualizing surveys like this allows users to explore whether a source of interest has already been observed by a survey. By executing a cone search and overlaying the found objects, one can check if sources visible in images have counterparts in specific catalogs, which can be beneficial when preparing a telescope proposal.


<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
## Disclaimers
Note that using the Astro Data Lab constitutes your agreement with our minimal <a href="https://datalab.noirlab.edu/about/disclaimers">Disclaimers</a>.

## Acknowledgments
If you use Astro Data Lab in your published research, please include the text in your paper's Acknowledgments section:

This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation.

If you use SPARCL jointly with the Astro Data Lab platform (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation.

In either case please cite the following papers:

Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097
If publishing in a AAS journal, also add the keyword: \facility{Astro Data Lab}

And if you are using SPARCL, please also add \software{SPARCL} and cite:

Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024 https://doi.org/10.48550/arXiv.2401.05576
The NOIRLab Library maintains lists of proper acknowledgments to use when publishing papers using the Lab's facilities, data, or services.

<a class="anchor" id="imports"></a>
# Imports and setup

In [ ]:
# Standard library
from getpass import getpass

# Third-party libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import astropy.units as u
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.wcs import WCS
from astropy.table import QTable
from astropy.coordinates import SkyCoord

from mocpy import MOC
from ipyaladin import Aladin
from sidecar import Sidecar

# Data Lab
from dl import authClient as ac, queryClient as qc

%matplotlib inline

<a class="anchor" id="Aladin"></a>
# Start Aladin Viewer

Let's start by opening an Aladin Lite viewer using SideCar. Running the following cell will open a new window to the right of where the notebook's cells are shown. 

Note: You do not need to use Sidecar to establish an Aladin Lite viewer session. If you were to just run the first line of the cell block, Aladin Lite would be opened below the code block. You could still interact with the viewer and utilize all of Aladin Lite's tools, but the session would remain below this code block, meaning you would need to scroll back up to this cell everytime you wanted to visit the viewer. 

In [ ]:
aladin = Aladin(full_screen=True)
with Sidecar(title="aladin_output",anchor='split-right'):
    display(aladin)

Next, let's adjust the field of view and center the Aladin viewer such that all three surveys are visible. 

In [ ]:
aladin.target = SkyCoord("12h04m22.76s", "-59d53m20.8s", frame="icrs")

aladin.fov = 180

<a class="anchor" id="Data"></a>
# Accessing the data

To create the MOCs that will be overlayed onto Aladin, we will use the <a href="https://cds-astro.github.io/mocpy/index.html">mocpy library</a>. But first, we need some data. For this tutorial, we will take advantage of some of the many data sets that are hosted by the Astro Data Lab. Specifically, we will be working with data from three surveys: The Survey of the MAgellanic Stellar History [(**SMASH**)](https://datalab.noirlab.edu/smash/smash.php), The Dark Energy Survey [(**DES**)](https://datalab.noirlab.edu/des/index.php), and The DECam Plane Survey [(**DECaPS DR2**)](https://datalab.noirlab.edu/decaps/index.php).

### Authentication

Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). However, some capacities, such as saving the results of your queries to your virtual storage space, require a login (i.e., you will need a registered user account).

If you need to log in to Data Lab, un-comment the first line of code in the cell below and execute it:

In [ ]:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
#print(token)
ac.whoAmI()

<a class="anchor" id="Query"></a>
# Querying the Astro Data Lab 

To access the data we will use to create the MOCs, we must draft some [ADQL](https://datalab.noirlab.edu/img/ADQL-20081030.pdf) queries like the following:

SELECT TOP 25000 ra, dec, gmag, random_id FROM smash_dr2.object WHERE random_id BETWEEN 25.0 AND 25.01

After running the query for SMASH, we will need to repeat the process for the DECaPS and DES surveys. If you find yourself getting stuck with SQL/ADQL, you can visit the following link to learn about common [SQL Gotchas](https://datalab.noirlab.edu/docs/manual/UsingAstroDataLab/SQLGotchas/index.html?highlight=sql).

In [ ]:
query = 'SELECT TOP 25000 ra, dec, mag_auto_g, random_id FROM des_dr2.main WHERE random_id BETWEEN 25.0 AND 25.01'
des = qc.query(sql = query, fmt = 'pandas')

# Let's add a column to the surveys
# Attach the survey name to each row
des['survey'] = 'des_dr2'

print('des_dr2 catalog')
print (des)

In [ ]:
query2 = 'SELECT TOP 25000 ra, dec, gmag, random_id FROM smash_dr2.object WHERE random_id BETWEEN 25.0 AND 25.01'
smash = qc.query(sql = query2, fmt = 'pandas')

smash['survey'] = 'smash_dr2'

print('smash_dr2')
print (smash)

In [ ]:
query3 = 'SELECT TOP 25000 ra, dec, mean_r, random_id FROM decaps_dr2.object WHERE random_id BETWEEN 25.0 AND 25.1'
decaps = qc.query(sql = query3, fmt = 'pandas')

decaps['survey'] = 'decaps_dr2'

print('decaps_dr2')
print (decaps)

<a class="anchor" id="prepare"></a>
# Prepare MOCs

Now that all three datasets have been downloaded, we can now create Multi-Order Coverage maps (MOCs) for each of them using the mocpy library. 

In [ ]:
def prepare_mocs(dataframe, max_norder):
    '''
    This function will perform two tasks:
    (1) Read in columns of a dataframe and output a dictionary that is compatible with Aladin Lite

    (2) Use the ra and dec info from the dictionary to create a MOC.
    '''
    # Define a dictionary
    col_dict = {}
    
    # Read in all of the column names and then add them to the dictionary.
    for c in dataframe.columns:
        col_dict[c] = dataframe[c].values
        
    # Now we are going to establish our MOC.  
    # Provide MOC.from_longlat() with ra and dec in degrees
    # max_norder:The depth of the smallest HEALPix cells contained in the MOC. Min = 1 | Max = 10
    moc = MOC.from_lonlat(
              col_dict['ra'].transpose() * u.deg,
              col_dict['dec'].transpose() * u.deg,
              max_norder=max_norder)

    return col_dict, moc

Before calling the prepare() function, we will need to create a list for each survey that contains the column names that we want to pass to Aladin. 

In [ ]:
des_dict, des_dr2_moc = prepare_mocs(des, max_norder=5)

smash_dict, smash_dr2_moc = prepare_mocs(smash, max_norder=5)

decaps_dict, decaps_dr2_moc = prepare_mocs(decaps, max_norder=5)

<a class="anchor" id="Plot"></a>
# Plot the MOCs

Before displaying the MOCs on Aladin, lets see what they look like using Matplotlib.

In [ ]:
def plot_mocs(moc, color, legend, title=""):
    fig = plt.figure(figsize=(15, 10))

    for c,m in enumerate(moc):
        wcs = WCS(naxis=2)
        wcs.wcs.ctype = ["GLON-AIT", "GLAT-AIT"] #Hammer-Aitoff projection
        wcs.wcs.crval = [300.0, 0.0]             #Specify the value of the reference pixel
        wcs.wcs.cdelt = [-0.675, 0.675]
        wcs.wcs.crpix = [240.5, 120.5]
    
        if c == 0:
            ax = fig.add_subplot(1, 1, 1, projection=wcs, frame_class=EllipticalFrame)
            patches = []
    
        m.fill(
            ax=ax,
            wcs=wcs,
            edgecolor=color[c],
            facecolor=color[c],
            linewidth=1.0,
            fill=True,
            alpha=0.5,
        )
        m.border(ax=ax, wcs=wcs, color="black", alpha=1)
    
        plt.xlabel("ra")
        plt.ylabel("dec")
        
        if title:
            plt.title(title)
        plt.grid(color="black", linestyle="dotted")
        
        patches.append(mpatches.Patch(color=color[c], label=legend[c]))
        plt.legend(handles=patches)
        
    plt.show()
    plt.close()

In [ ]:
moc = [des_dr2_moc, smash_dr2_moc, decaps_dr2_moc]
legend = ['des_dr2_moc', 'smash_dr2_moc', 'decaps_dr2_moc']
colors = ['cyan', 'red', 'blue']
plot_mocs(moc,title='',color=colors,legend=legend)

<a class="anchor" id="AladinMOCs"></a>
# Add the MOC to Aladin

Now that we have seen the survey coverage using Matplotlib, let's use ipyaladin's add_moc() function to overlay the MOCs onto Aladin.

In [ ]:
aladin.add_moc(des_dr2_moc, color='cyan',   name='des_dr2_moc', opacity=0.4)
aladin.add_moc(smash_dr2_moc, color='red',  name='smash_dr2_moc', opacity=0.4)
aladin.add_moc(decaps_dr2_moc, color='blue', name='decaps_dr2_moc', opacity=0.4)

By now, all three MOCs are displayed on the Aladin viewer. Take the time to zoom in and pan around to view the areas where the different surveys overlap. 

<a class="anchor" id="source"></a>
# Display individual sources on Aladin

Now, let's plot the individual sources onto Aladin. Currently, Aladin Lite only works reliably with QTables, so we are going to reformat our pandas data frames into something the Aladin viewer will accept. 

In [ ]:
#des_dr2
des_table    = QTable([des_dict['ra'],des_dict['dec'],des.index,\
                      des_dict['mag_auto_g'],des_dict['survey']],\
                      names=["ra","dec","index","mag_auto_g","survey"])

#smash_dr2
smash_table  = QTable([smash_dict['ra'],smash_dict['dec'],smash.index,\
                      smash_dict['gmag'],smash_dict['survey']],\
                      names=["ra","dec","index","gmag","survey"])

#decaps_dr2
decaps_table = QTable([decaps_dict['ra'],decaps_dict['dec'],decaps.index,\
                      decaps_dict['mean_r'],decaps_dict['survey']],\
                      names=["ra","dec","index","mean_r","survey"])

With our datasets reformatted, we can add now display on the Aladin viewer using the add_table() function.  

In [ ]:
aladin.add_table(des_table,name='des_dr2',color='cyan')
aladin.add_table(smash_table,name='smash_dr2',color='red')
aladin.add_table(decaps_table,name='decaps_dr2',color='blue')

By now, your Aladin viewer maybe hard to read since there are many MOCs and individual data points plotted ontop of each other. You can remove some of these objects by clicking the 'Overlays menu' in your Aladin viewer. A dropdown menu will appear displaying the names of all the MOCs and data tables currently being displayed. From there, you can hover your mouse over any item and either click on the 'eye' icon to hide the object, or the 'trash' icon to fully remove it from the viewer. 

![Pointing out the location of the hide and remove icons in Aladin Lite.](Remove_layer.png "Remove layer.")